In [ ]:
!pip install torch torchvision opencv-python pillow requests


In [ ]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import os
import time
from datetime import datetime
import tkinter as tk
from tkinter import filedialog, messagebox
import threading

# ==================== CONFIGURATION ====================
MODEL_PATH = '/home/gess/Documents/eye_detection_model/professional_eye_detector.pth'
IMG_SIZE = (64, 64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==================== EXACT MODEL ARCHITECTURE ====================
class ExactEyeDetectionModel(nn.Module):
    def __init__(self):
        super(ExactEyeDetectionModel, self).__init__()
        
        # EXACT architecture matching the trained model
        self.features = nn.Sequential(
            # Block 1 - EXACT sizes
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
            
            # Block 2 - EXACT sizes  
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
            
            # Block 3 - EXACT sizes
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.25),
            
            # Global pooling - EXACT
            nn.AdaptiveAvgPool2d((4, 4)),
        )
        
        # Classifier - EXACT sizes (128 * 4 * 4 = 2048)
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),  # EXACT: 2048 -> 256
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ==================== SIMPLE & FAST EYE DETECTION ====================
class SimpleEyeDetection:
    def __init__(self):
        self.model = None
        self.model_path = MODEL_PATH
        self.load_model()
        
        # Face detection
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
        
        # Simple drowsiness detection
        self.eye_close_counter = 0
        self.eye_close_threshold = 8
        
        print("✅ Simple Eye Detection Initialized!")
    
    def load_model(self):
        """Load model with EXACT architecture"""
        try:
            if not os.path.exists(self.model_path):
                # Try to find model
                alt_paths = [
                    '/home/gess/Documents/sub/Py/hhehee/eye_detection_model/professional_eye_detector.pth',
                    './professional_eye_detector.pth',
                    'eye_detection_model/professional_eye_detector.pth',
                ]
                for path in alt_paths:
                    if os.path.exists(path):
                        self.model_path = path
                        print(f"✅ Found model at: {path}")
                        break
            
            if os.path.exists(self.model_path):
                checkpoint = torch.load(self.model_path, map_location=device)
                self.model = ExactEyeDetectionModel().to(device)
                
                # Load state dict
                if 'model_state_dict' in checkpoint:
                    state_dict = checkpoint['model_state_dict']
                else:
                    state_dict = checkpoint
                
                # Load exactly - should work now
                self.model.load_state_dict(state_dict)
                self.model.eval()
                print("🎯 Model loaded EXACTLY!")
                return True
            else:
                print("❌ Model file not found")
                return False
                
        except Exception as e:
            print(f"❌ Model loading failed: {e}")
            return False
    
    def fast_preprocess(self, eye_img):
        """Fast preprocessing"""
        try:
            if eye_img is None or eye_img.size == 0:
                return None
            
            # Just resize and normalize
            resized = cv2.resize(eye_img, IMG_SIZE)
            normalized = resized.astype('float32') / 255.0
            
            return np.expand_dims(normalized, axis=0)
            
        except:
            return None
    
    def predict_eye(self, eye_image):
        """Fast prediction"""
        if self.model is None:
            return None, 0.0
        
        try:
            processed = self.fast_preprocess(eye_image)
            if processed is None:
                return None, 0.0
            
            input_tensor = torch.tensor(processed, dtype=torch.float32).unsqueeze(0).to(device)
            
            with torch.no_grad():
                output = self.model(input_tensor)
                confidence = output.item()
                prediction = 1 if confidence > 0.5 else 0
            
            return prediction, confidence
            
        except:
            return None, 0.0
    
    def extract_eyes_simple(self, image):
        """Simple eye extraction"""
        try:
            if len(image.shape) == 3:
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                gray = image
            
            eyes = []
            positions = []
            
            # Simple face detection
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)
            
            for (x, y, w, h) in faces:
                face_roi = gray[y:y+h, x:x+w]
                
                # Simple eye detection
                detected_eyes = self.eye_cascade.detectMultiScale(face_roi, 1.1, 4)
                
                for (ex, ey, ew, eh) in detected_eyes[:2]:  # Max 2 eyes
                    if ew >= 20 and eh >= 10:
                        eye_roi = face_roi[ey:ey+eh, ex:ex+ew]
                        eyes.append(eye_roi)
                        positions.append((x+ex, y+ey, ew, eh))
            
            return eyes, positions
            
        except:
            return [], []
    
    def analyze_simple(self, eye_states):
        """Simple drowsiness analysis"""
        if not eye_states:
            return "NO EYES", 0
        
        # Count closed eyes
        closed_count = sum(1 for state in eye_states if state == 0)
        
        # Simple logic: if more than half eyes are closed
        if closed_count > len(eye_states) / 2:
            self.eye_close_counter += 1
        else:
            self.eye_close_counter = max(0, self.eye_close_counter - 1)
        
        # Determine status
        if self.eye_close_counter >= self.eye_close_threshold:
            return "🚨 DROWSY!", self.eye_close_counter
        elif self.eye_close_counter > self.eye_close_threshold * 0.5:
            return "⚠️ TIRED", self.eye_close_counter
        else:
            return "✅ AWAKE", self.eye_close_counter
    
    def process_frame_simple(self, frame, draw=True):
        """Simple frame processing"""
        try:
            # Extract eyes
            eyes, positions = self.extract_eyes_simple(frame)
            
            if not eyes:
                if draw:
                    self.draw_simple_info(frame, "NO EYES", 0, 0)
                return frame, [], "NO EYES"
            
            # Predict each eye
            eye_states = []
            for eye_img in eyes:
                pred, _ = self.predict_eye(eye_img)
                if pred is not None:
                    eye_states.append(pred)
            
            # Analyze
            status, counter = self.analyze_simple(eye_states)
            
            # Draw if requested
            if draw:
                self.draw_simple_detections(frame, positions, eye_states)
                self.draw_simple_info(frame, status, len(eyes), counter)
            
            return frame, eye_states, status
            
        except Exception as e:
            return frame, [], "ERROR"
    
    def draw_simple_detections(self, frame, positions, eye_states):
        """Simple drawing"""
        for i, ((x, y, w, h), state) in enumerate(zip(positions, eye_states)):
            color = (0, 255, 0) if state == 1 else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            
            # Simple label
            label = "O" if state == 1 else "C"
            cv2.putText(frame, label, (x, y-5), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    def draw_simple_info(self, frame, status, eyes_count, counter):
        """Simple info display"""
        # Status color
        if "🚨" in status:
            color = (0, 0, 255)  # Red
        elif "⚠️" in status:
            color = (0, 255, 255)  # Yellow
        else:
            color = (0, 255, 0)  # Green
        
        # Simple text
        cv2.putText(frame, status, (10, 30), 
                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Eyes: {eyes_count}", (10, 60), 
                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(frame, f"Count: {counter}", (10, 80), 
                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    def try_camera_simple(self):
        """Simple camera detection"""
        # Try different camera indices
        for i in range(5):
            cap = cv2.VideoCapture(i)
            if cap.isOpened():
                ret, frame = cap.read()
                if ret:
                    cap.release()
                    print(f"📷 Using camera {i}")
                    return i
            cap.release()
        
        print("❌ No camera found")
        return -1
    
    def webcam_simple(self):
        """Simple webcam detection"""
        camera_index = self.try_camera_simple()
        if camera_index == -1:
            messagebox.showerror("Camera Error", 
                               "No camera found!\n\n"
                               "Please check:\n"
                               "1. Camera is connected\n"
                               "2. Camera drivers are installed\n"
                               "3. No other app is using camera")
            return False
        
        cap = cv2.VideoCapture(camera_index)
        
        # Set lower resolution for speed
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        print("🎥 Starting simple webcam...")
        print("💡 Press 'Q' to quit, 'R' to reset counter")
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Mirror frame
                frame = cv2.flip(frame, 1)
                
                # Process frame
                processed_frame, _, status = self.process_frame_simple(frame)
                
                # Display
                cv2.imshow('Simple Eye Detection', processed_frame)
                
                # Handle keys
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('r'):
                    self.eye_close_counter = 0
                    print("🔄 Counter reset")
                
        except Exception as e:
            print(f"Webcam error: {e}")
        
        finally:
            cap.release()
            cv2.destroyAllWindows()
            print("✅ Webcam stopped")
        
        return True
    
    def process_image_simple(self, image_path):
        """Simple image processing"""
        if not os.path.exists(image_path):
            return False, "Image not found"
        
        image = cv2.imread(image_path)
        if image is None:
            return False, "Cannot load image"
        
        eyes, positions = self.extract_eyes_simple(image)
        
        if not eyes:
            return False, "No eyes detected"
        
        predictions = []
        for eye_img in eyes:
            pred, conf = self.predict_eye(eye_img)
            if pred is not None:
                predictions.append(pred)
        
        if not predictions:
            return False, "Prediction failed"
        
        open_count = sum(predictions)
        closed_count = len(predictions) - open_count
        state = "OPEN" if open_count > closed_count else "CLOSED"
        
        # Show result
        result_frame, _, _ = self.process_frame_simple(image)
        cv2.imshow('Image Result', result_frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        result = {
            'state': state,
            'eyes_detected': len(eyes),
            'open_eyes': open_count,
            'closed_eyes': closed_count
        }
        
        return True, result

# ==================== CLEAN GUI ====================
class CleanEyeDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("👁️ Eye Detection")
        self.root.geometry("400x300")
        
        self.detector = SimpleEyeDetection()
        self.setup_gui()
    
    def setup_gui(self):
        """Clean and simple GUI"""
        # Title
        title = tk.Label(self.root, text="👁️ EYE DETECTION", 
                        font=('Arial', 16, 'bold'))
        title.pack(pady=20)
        
        # Status
        self.status = tk.Label(self.root, text="Ready to use", 
                              font=('Arial', 12))
        self.status.pack(pady=10)
        
        # Buttons frame
        btn_frame = tk.Frame(self.root)
        btn_frame.pack(pady=20)
        
        # Webcam button
        webcam_btn = tk.Button(btn_frame, text="🎥 Start Webcam", 
                             command=self.start_webcam,
                             font=('Arial', 12), bg='#4CAF50', fg='white',
                             width=15, height=2)
        webcam_btn.grid(row=0, column=0, padx=10, pady=10)
        
        # Image button
        image_btn = tk.Button(btn_frame, text="🖼️ Open Image", 
                            command=self.open_image,
                            font=('Arial', 12), bg='#2196F3', fg='white',
                            width=15, height=2)
        image_btn.grid(row=0, column=1, padx=10, pady=10)
        
        # Reset button
        reset_btn = tk.Button(btn_frame, text="🔄 Reset", 
                            command=self.reset_counter,
                            font=('Arial', 12), bg='#FF9800', fg='white',
                            width=15, height=2)
        reset_btn.grid(row=1, column=0, padx=10, pady=10)
        
        # Exit button
        exit_btn = tk.Button(btn_frame, text="🚪 Exit", 
                           command=self.root.quit,
                           font=('Arial', 12), bg='#f44336', fg='white',
                           width=15, height=2)
        exit_btn.grid(row=1, column=1, padx=10, pady=10)
        
        # Info
        info = tk.Label(self.root, text="Press 'Q' to quit webcam, 'R' to reset counter",
                       font=('Arial', 9), fg='gray')
        info.pack(pady=10)
    
    def start_webcam(self):
        """Start webcam in thread"""
        def run_webcam():
            self.status.config(text="Starting webcam...", fg='orange')
            success = self.detector.webcam_simple()
            if success:
                self.status.config(text="Webcam finished", fg='green')
            else:
                self.status.config(text="Webcam failed", fg='red')
        
        threading.Thread(target=run_webcam, daemon=True).start()
    
    def open_image(self):
        """Open and process image"""
        file_path = filedialog.askopenfilename(
            title="Select Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")]
        )
        
        if file_path:
            def process():
                self.status.config(text="Processing image...", fg='orange')
                success, result = self.detector.process_image_simple(file_path)
                
                if success:
                    self.status.config(text="Image processed", fg='green')
                    messagebox.showinfo("Result", 
                                      f"State: {result['state']}\n"
                                      f"Eyes detected: {result['eyes_detected']}\n"
                                      f"Open eyes: {result['open_eyes']}\n"
                                      f"Closed eyes: {result['closed_eyes']}")
                else:
                    self.status.config(text="Image failed", fg='red')
                    messagebox.showerror("Error", result)
            
            threading.Thread(target=process, daemon=True).start()
    
    def reset_counter(self):
        """Reset drowsiness counter"""
        self.detector.eye_close_counter = 0
        self.status.config(text="Counter reset", fg='blue')
        messagebox.showinfo("Reset", "Drowsiness counter has been reset!")

# ==================== DIRECT TEST (without GUI) ====================
def test_system():
    """Test the system directly"""
    print("🧪 Testing Eye Detection System...")
    
    detector = SimpleEyeDetection()
    if not detector.model:
        print("❌ TEST FAILED: Model not loaded")
        return False
    
    print("✅ Model loaded successfully!")
    
    # Test with webcam
    print("🎥 Testing webcam...")
    webcam_works = detector.webcam_simple()
    
    if webcam_works:
        print("✅ Webcam test PASSED")
    else:
        print("❌ Webcam test FAILED")
    
    return webcam_works

# ==================== MAIN ====================
def main():
    print("🚀 Starting Simple Eye Detection System...")
    
    # Test if model loads
    detector = SimpleEyeDetection()
    if not detector.model:
        print("❌ Cannot start: Model loading failed")
        print("💡 Please check:")
        print("   1. Model file exists")
        print("   2. File path is correct") 
        print("   3. PyTorch is properly installed")
        return
    
    print("✅ System ready!")
    
    # Start GUI
    root = tk.Tk()
    app = CleanEyeDetectionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

In [2]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import os
import time

# ==================== CONFIGURATION ====================
MODEL_PATH = '/home/gess/Documents/eye_detection_model/professional_eye_detector.pth'
IMG_SIZE = (64, 64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==================== BIAS-AWARE MODEL ====================
class BiasAwareEyeModel(nn.Module):
    def __init__(self):
        super(BiasAwareEyeModel, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            nn.AdaptiveAvgPool2d((4, 4)),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ==================== BIAS CORRECTION SYSTEM ====================
class BiasCorrectedDetection:
    def __init__(self):
        self.model = None
        self.load_model()
        
        # Face and eye detection
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
        
        # Bias correction
        self.prediction_history = []
        self.history_size = 5
        self.confidence_threshold = 0.7  # Higher threshold for certainty
        
        # Dynamic threshold for bias correction
        self.dynamic_threshold = 0.5
        self.adjustment_factor = 0.1
        
        print("🎯 Bias-Corrected Eye Detection Ready!")
    
    def load_model(self):
        """Load model with bias awareness"""
        try:
            if not os.path.exists(MODEL_PATH):
                print(f"❌ Model not found: {MODEL_PATH}")
                return False
            
            checkpoint = torch.load(MODEL_PATH, map_location=device)
            print("✅ Checkpoint loaded")
            
            self.model = BiasAwareEyeModel().to(device)
            
            if 'model_state_dict' in checkpoint:
                self.model.load_state_dict(checkpoint['model_state_dict'], strict=False)
            else:
                self.model.load_state_dict(checkpoint, strict=False)
            
            self.model.eval()
            print("✅ Model loaded with bias correction")
            return True
            
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return False
    
    def analyze_eye_features(self, eye_img):
        """Analyze eye image features to help with bias correction"""
        try:
            if eye_img is None:
                return None
            
            # Calculate image statistics that might indicate closed eyes
            mean_intensity = np.mean(eye_img)
            std_intensity = np.std(eye_img)
            
            # Closed eyes often have lower contrast and different intensity distribution
            contrast = std_intensity / (mean_intensity + 1e-6)
            
            # Calculate percentage of dark pixels (potential closed eyes)
            dark_pixels = np.sum(eye_img < 50) / eye_img.size
            
            features = {
                'mean_intensity': mean_intensity,
                'contrast': contrast,
                'dark_pixels': dark_pixels
            }
            
            return features
            
        except:
            return None
    
    def bias_aware_preprocess(self, eye_img):
        """Preprocessing with bias consideration"""
        try:
            if eye_img is None or eye_img.size == 0:
                return None
            
            # Resize
            resized = cv2.resize(eye_img, IMG_SIZE)
            
            # Apply CLAHE for better contrast - especially helpful for closed eyes
            clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
            enhanced = clahe.apply(resized)
            
            # Normalize
            normalized = enhanced.astype('float32') / 255.0
            
            return np.expand_dims(normalized, axis=0)
            
        except:
            return None
    
    def adjust_prediction_with_features(self, prediction, confidence, features):
        """Adjust prediction based on image features"""
        if features is None:
            return prediction, confidence
        
        # If image is very dark and low contrast, likely closed eyes
        if features['mean_intensity'] < 80 and features['contrast'] < 0.5:
            if prediction == 1:  # If model says open but features say closed
                if confidence < 0.8:  # Only adjust if not very confident
                    return 0, 1 - confidence  # Flip prediction
                    
        # If many dark pixels, might be closed eyes
        if features['dark_pixels'] > 0.3 and confidence < 0.7:
            if prediction == 1:
                return 0, 1 - confidence
                
        return prediction, confidence
    
    def predict_with_bias_correction(self, eye_image):
        """Predict with bias correction mechanisms"""
        if self.model is None:
            return None, 0.0, "NO MODEL"
        
        try:
            # Preprocess
            processed = self.bias_aware_preprocess(eye_image)
            if processed is None:
                return None, 0.0, "PREPROCESS FAILED"
            
            # Get image features for bias correction
            features = self.analyze_eye_features(eye_image)
            
            # Model prediction
            input_tensor = torch.tensor(processed, dtype=torch.float32).unsqueeze(0).to(device)
            
            with torch.no_grad():
                output = self.model(input_tensor)
                raw_confidence = output.item()
                
                # ADJUSTABLE THRESHOLD - try different values
                current_threshold = self.dynamic_threshold
                raw_prediction = 1 if raw_confidence > current_threshold else 0
                
                # Apply bias correction based on image features
                final_prediction, final_confidence = self.adjust_prediction_with_features(
                    raw_prediction, raw_confidence, features
                )
                
                # Update dynamic threshold based on confidence
                if final_confidence > 0.8:
                    # Good confidence, keep threshold
                    pass
                elif final_confidence < 0.6:
                    # Low confidence, adjust threshold
                    self.dynamic_threshold += 0.05
                else:
                    # Medium confidence, slight adjustment
                    self.dynamic_threshold += 0.02
                
                # Keep threshold in reasonable range
                self.dynamic_threshold = max(0.3, min(0.7, self.dynamic_threshold))
                
                # Store prediction for temporal smoothing
                self.prediction_history.append(final_prediction)
                if len(self.prediction_history) > self.history_size:
                    self.prediction_history.pop(0)
                
                # Apply temporal smoothing (majority vote from recent frames)
                if len(self.prediction_history) >= 3:
                    recent_majority = 1 if sum(self.prediction_history) > len(self.prediction_history) / 2 else 0
                    if recent_majority != final_prediction and final_confidence < 0.7:
                        final_prediction = recent_majority
                        final_confidence = 0.6  # Moderate confidence for smoothed prediction
            
            return final_prediction, final_confidence, "SUCCESS"
            
        except Exception as e:
            return None, 0.0, f"ERROR: {e}"
    
    def extract_eyes_robust(self, image):
        """Robust eye extraction with multiple attempts"""
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            eyes = []
            positions = []
            
            # Improve face detection with better parameters
            faces = self.face_cascade.detectMultiScale(
                gray, 
                scaleFactor=1.05,
                minNeighbors=6,
                minSize=(50, 50),
                flags=cv2.CASCADE_SCALE_IMAGE
            )
            
            for (x, y, w, h) in faces:
                face_roi = gray[y:y+h, x:x+w]
                
                # Try multiple eye detection parameters
                eye_params = [
                    {'scale': 1.05, 'neighbors': 3, 'minSize': (20, 10)},  # Sensitive
                    {'scale': 1.1, 'neighbors': 5, 'minSize': (25, 15)},   # Balanced
                    {'scale': 1.02, 'neighbors': 2, 'minSize': (15, 8)},   # Very sensitive
                ]
                
                all_eyes = []
                for params in eye_params:
                    detected = self.eye_cascade.detectMultiScale(
                        face_roi,
                        scaleFactor=params['scale'],
                        minNeighbors=params['neighbors'],
                        minSize=params['minSize']
                    )
                    all_eyes.extend(detected)
                
                # Remove duplicates and take best 2 eyes
                unique_eyes = []
                for eye in all_eyes:
                    if len(unique_eyes) >= 2:
                        break
                    # Simple duplicate check
                    is_duplicate = False
                    for u_eye in unique_eyes:
                        if abs(eye[0] - u_eye[0]) < 10 and abs(eye[1] - u_eye[1]) < 10:
                            is_duplicate = True
                            break
                    if not is_duplicate:
                        unique_eyes.append(eye)
                
                for (ex, ey, ew, eh) in unique_eyes:
                    eye_roi = face_roi[ey:ey+eh, ex:ex+ew]
                    eyes.append(eye_roi)
                    positions.append((x+ex, y+ey, ew, eh))
            
            return eyes, positions
            
        except Exception as e:
            return [], []
    
    def process_frame_with_correction(self, frame):
        """Process frame with comprehensive bias correction"""
        try:
            eyes, positions = self.extract_eyes_robust(frame)
            
            predictions = []
            states = []
            
            for i, (eye_img, (x, y, w, h)) in enumerate(zip(eyes, positions)):
                prediction, confidence, status = self.predict_with_bias_correction(eye_img)
                
                if prediction is not None:
                    state = "OPEN" if prediction == 1 else "CLOSED"
                    display_confidence = confidence if prediction == 1 else 1 - confidence
                    
                    predictions.append((state, display_confidence))
                    states.append(prediction)
                    
                    # Visual feedback with confidence indication
                    if confidence > 0.8:
                        color = (0, 255, 0) if state == "OPEN" else (0, 0, 255)
                        thickness = 3
                    elif confidence > 0.6:
                        color = (0, 200, 0) if state == "OPEN" else (0, 0, 200) 
                        thickness = 2
                    else:
                        color = (0, 150, 0) if state == "OPEN" else (0, 0, 150)
                        thickness = 1
                    
                    cv2.rectangle(frame, (x, y), (x+w, y+h), color, thickness)
                    
                    # Text with confidence
                    text = f"{state} ({display_confidence:.2f})"
                    cv2.putText(frame, text, (x, y-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            return frame, predictions
            
        except Exception as e:
            return frame, []
    
    def run_bias_corrected_webcam(self):
        """Run webcam with bias correction"""
        # Find camera
        for i in range(5):
            cap = cv2.VideoCapture(i)
            if cap.isOpened():
                ret, _ = cap.read()
                if ret:
                    break
            cap.release()
        else:
            print("❌ No camera found!")
            return
        
        print(f"🎥 Starting bias-corrected detection on camera {i}...")
        print("🔧 Features enabled:")
        print("   • Dynamic threshold adjustment")
        print("   • Image feature analysis") 
        print("   • Temporal smoothing")
        print("   • Confidence-based correction")
        print("💡 Press 'Q' to quit, 'T' to toggle threshold")
        
        threshold_display = True
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                frame = cv2.flip(frame, 1)
                
                # Process with bias correction
                processed_frame, predictions = self.process_frame_with_correction(frame)
                
                # Display current threshold
                if threshold_display:
                    cv2.putText(processed_frame, f"Threshold: {self.dynamic_threshold:.2f}", 
                              (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
                
                cv2.putText(processed_frame, "Press 'Q' to quit, 'T' toggle threshold", 
                          (10, processed_frame.shape[0]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
                
                cv2.imshow('Bias-Corrected Eye Detection', processed_frame)
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('t'):
                    threshold_display = not threshold_display
                elif key == ord('+'):
                    self.dynamic_threshold = min(0.7, self.dynamic_threshold + 0.05)
                elif key == ord('-'):
                    self.dynamic_threshold = max(0.3, self.dynamic_threshold - 0.05)
                    
        except Exception as e:
            print(f"Error: {e}")
        
        finally:
            cap.release()
            cv2.destroyAllWindows()
            print("✅ Bias-corrected detection stopped")

# ==================== QUICK BIAS TEST ====================
def quick_bias_test():
    """Quick test to check model bias"""
    print("🧪 Running Quick Bias Test...")
    
    detector = BiasCorrectedDetection()
    if not detector.model:
        print("❌ Cannot test without model")
        return
    
    # Test with sample patterns that should be closed eyes
    test_cases = [
        ("Dark image", np.zeros((50, 50), dtype=np.uint8)),
        ("Low contrast", np.full((50, 50), 100, dtype=np.uint8)),
        ("Mixed", np.random.randint(0, 50, (50, 50), dtype=np.uint8)),
    ]
    
    for name, test_img in test_cases:
        prediction, confidence, status = detector.predict_with_bias_correction(test_img)
        if prediction is not None:
            state = "OPEN" if prediction == 1 else "CLOSED"
            print(f"   {name}: {state} (conf: {confidence:.3f}) - {status}")
        else:
            print(f"   {name}: FAILED - {status}")

# ==================== MAIN ====================
def main():
    print("🎯 BIAS-CORRECTED EYE DETECTION SYSTEM")
    print("=" * 50)
    
    # Quick bias test
    quick_bias_test()
    
    # Main system
    detector = BiasCorrectedDetection()
    if not detector.model:
        print("❌ Cannot start without model")
        return
    
    print("\n🚀 Starting main system...")
    detector.run_bias_corrected_webcam()

if __name__ == "__main__":
    main()

🎯 BIAS-CORRECTED EYE DETECTION SYSTEM
🧪 Running Quick Bias Test...
✅ Checkpoint loaded
❌ Error loading model: Error(s) in loading state_dict for BiasAwareEyeModel:
	size mismatch for features.5.weight: copying a param with shape torch.Size([64, 32, 3, 3]) from checkpoint, the shape in current model is torch.Size([64]).
🎯 Bias-Corrected Eye Detection Ready!
   Dark image: CLOSED (conf: 0.476) - SUCCESS
   Low contrast: OPEN (conf: 0.617) - SUCCESS
   Mixed: CLOSED (conf: 0.301) - SUCCESS
✅ Checkpoint loaded
❌ Error loading model: Error(s) in loading state_dict for BiasAwareEyeModel:
	size mismatch for features.5.weight: copying a param with shape torch.Size([64, 32, 3, 3]) from checkpoint, the shape in current model is torch.Size([64]).
🎯 Bias-Corrected Eye Detection Ready!

🚀 Starting main system...
🎥 Starting bias-corrected detection on camera 0...
🔧 Features enabled:
   • Dynamic threshold adjustment
   • Image feature analysis
   • Temporal smoothing
   • Confidence-based correction

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import annotations

import os
import sys
import time
from collections import deque
from pathlib import Path
from typing import Deque, Tuple, Optional

import cv2
import numpy as np
from ultralytics import YOLO

# ===========================
# CẤU HÌNH ĐƯỜNG DẪN & THAM SỐ
# ===========================

DATA_ROOT = Path("/home/gess/Documents/sub/TrainModel")
MODEL_DIR = DATA_ROOT / "models"
BEST_MODEL = MODEL_DIR / "best_drowsy.pt"  # đã train xong ở train_drowsy.py

# Class name phải trùng lúc train
CLS_OPEN = "open_eye"
CLS_CLOSED = "closed_eye"

# Tham số drowsy
HISTORY_LEN = 30        # số frame lưu history (khoảng 1 giây nếu 30fps)
DROWSY_THRESHOLD = 0.6  # tỉ lệ closed_eye / (open+closed) trong history
MIN_EYES_PER_FRAME = 1  # số mắt min để frame được tính

# Confidence ngưỡng để tính
CONF_THRESH = 0.5

# ===========================
# HÀM TIỆN ÍCH
# ===========================

def load_model(model_path: Path) -> YOLO:
    if not model_path.exists():
        print(f"[❌] Không tìm thấy model: {model_path}")
        sys.exit(1)
    print(f"[🚀] Load model: {model_path}")
    model = YOLO(str(model_path))
    return model


def decode_counts_from_result(
    result,
    cls_open: str = CLS_OPEN,
    cls_closed: str = CLS_CLOSED,
    conf_thresh: float = CONF_THRESH,
) -> Tuple[int, int]:
    """
    Đếm số open_eye và closed_eye trong 1 kết quả YOLO.

    Return:
        (num_open, num_closed)
    """
    names = result.names
    num_open = 0
    num_closed = 0

    if result.boxes is None or len(result.boxes) == 0:
        return 0, 0

    for b in result.boxes:
        cls_id = int(b.cls[0].item())
        conf = float(b.conf[0].item())
        if conf < conf_thresh:
            continue
        cls_name = names.get(cls_id, str(cls_id))
        if cls_name == cls_open:
            num_open += 1
        elif cls_name == cls_closed:
            num_closed += 1
    return num_open, num_closed


def update_drowsy_history(
    history: Deque[Tuple[int, int]],
    num_open: int,
    num_closed: int,
    maxlen: int = HISTORY_LEN,
) -> Tuple[float, int, int]:
    """
    Cập nhật history (deque) với (open, closed) của frame mới.
    Trả về:
        - closed_ratio (tỉ lệ closed / total)
        - total_open
        - total_closed
    """
    history.append((num_open, num_closed))
    if len(history) > maxlen:
        history.popleft()

    total_open = sum(o for o, _ in history)
    total_closed = sum(c for _, c in history)
    total = total_open + total_closed
    closed_ratio = (total_closed / total) if total > 0 else 0.0
    return closed_ratio, total_open, total_closed


def draw_info(
    frame: np.ndarray,
    num_open: int,
    num_closed: int,
    closed_ratio: float,
    is_drowsy: bool,
) -> np.ndarray:
    """
    Vẽ thông tin lên frame: số mắt mở/nhắm, tỉ lệ, trạng thái drowsy.
    """
    h, w = frame.shape[:2]
    overlay = frame.copy()

    # Bar mờ phía trên
    cv2.rectangle(overlay, (0, 0), (w, 80), (0, 0, 0), -1)
    alpha = 0.4
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    # Text info
    base_y = 25
    cv2.putText(
        frame,
        f"Open: {num_open}  Closed: {num_closed}",
        (10, base_y),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (0, 255, 255),
        2,
        cv2.LINE_AA,
    )
    cv2.putText(
        frame,
        f"Closed ratio: {closed_ratio:.2f}",
        (10, base_y + 25),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (255, 255, 0),
        2,
        cv2.LINE_AA,
    )

    status_text = "DROWSY!" if is_drowsy else "OK"
    color = (0, 0, 255) if is_drowsy else (0, 255, 0)
    cv2.putText(
        frame,
        f"Status: {status_text}",
        (10, base_y + 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        color,
        2,
        cv2.LINE_AA,
    )

    # Nếu drowsy thì vẽ khung cảnh báo
    if is_drowsy:
        cv2.rectangle(frame, (0, 80), (w - 1, h - 1), (0, 0, 255), 4)

    return frame


# ===========================
# WEBCAM REALTIME
# ===========================

def run_webcam(
    model: YOLO,
    cam_index: int = 0,
    history_len: int = HISTORY_LEN,
    drowsy_thresh: float = DROWSY_THRESHOLD,
):
    print(f"[🎥] Đang mở webcam {cam_index}...")
    cap = cv2.VideoCapture(cam_index)
    if not cap.isOpened():
        print("[❌] Không mở được webcam, thử index khác (0,1,2,...) hoặc kiểm tra quyền.")
        return

    history: Deque[Tuple[int, int]] = deque(maxlen=history_len)

    print("[INFO] Nhấn 'q' để thoát.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("[❌] Không đọc được frame từ webcam.")
            break

        # YOLO expect BGR → OK
        results = model.predict(
            source=frame,
            imgsz=768,
            conf=CONF_THRESH,
            verbose=False,
        )

        if len(results) == 0:
            num_open, num_closed = 0, 0
        else:
            num_open, num_closed = decode_counts_from_result(results[0])

        closed_ratio, total_open, total_closed = update_drowsy_history(
            history, num_open, num_closed, maxlen=history_len
        )

        is_drowsy = False
        total = total_open + total_closed
        if total >= MIN_EYES_PER_FRAME and closed_ratio >= drowsy_thresh:
            is_drowsy = True

        # Vẽ detection box của YOLO
        plotted = results[0].plot() if len(results) > 0 else frame.copy()

        # Vẽ info Drowsy
        plotted = draw_info(plotted, num_open, num_closed, closed_ratio, is_drowsy)

        cv2.imshow("Drowsy Detection - Webcam", plotted)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("[✅] Đã thoát realtime.")


# ===========================
# PHÂN TÍCH 1 ẢNH
# ===========================

def analyze_image(model: YOLO, img_path: str | Path):
    img_path = Path(img_path)
    if not img_path.exists():
        print(f"[❌] Không tìm thấy ảnh: {img_path}")
        return

    img = cv2.imread(str(img_path))
    if img is None:
        print(f"[❌] Không đọc được ảnh: {img_path}")
        return

    results = model.predict(
        source=img,
        imgsz=768,
        conf=CONF_THRESH,
        verbose=False,
    )
    if len(results) == 0:
        print("[ℹ] Không có detection nào.")
        cv2.imshow("Drowsy - Image", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return

    r = results[0]
    num_open, num_closed = decode_counts_from_result(r)
    total = num_open + num_closed
    closed_ratio = (num_closed / total) if total > 0 else 0.0
    is_drowsy = closed_ratio >= DROWSY_THRESHOLD and total >= MIN_EYES_PER_FRAME

    print("===== PHÂN TÍCH ẢNH =====")
    print(f"Ảnh       : {img_path}")
    print(f"Open eyes : {num_open}")
    print(f"Closed    : {num_closed}")
    print(f"Closed ratio: {closed_ratio:.2f}")
    print(f"Trạng thái: {'DROWSY' if is_drowsy else 'OK'}")

    plotted = r.plot()
    plotted = draw_info(plotted, num_open, num_closed, closed_ratio, is_drowsy)
    cv2.imshow("Drowsy - Image", plotted)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# ===========================
# PHÂN TÍCH FOLDER ẢNH
# ===========================

def analyze_folder(model: YOLO, folder: str | Path):
    folder = Path(folder)
    if not folder.exists():
        print(f"[❌] Không tìm thấy folder: {folder}")
        return

    exts = [".jpg", ".jpeg", ".png", ".bmp"]
    img_files = sorted(
        [p for p in folder.rglob("*") if p.suffix.lower() in exts]
    )
    if not img_files:
        print(f"[ℹ] Folder không có ảnh hợp lệ: {folder}")
        return

    print(f"[📂] Tìm thấy {len(img_files)} ảnh trong {folder}")
    for img_path in img_files:
        print(f"\n--- {img_path} ---")
        analyze_image(model, img_path)


# ===========================
# PHÂN TÍCH VIDEO
# ===========================

def analyze_video(
    model: YOLO,
    video_path: str | Path,
    history_len: int = HISTORY_LEN,
    drowsy_thresh: float = DROWSY_THRESHOLD,
):
    video_path = Path(video_path)
    if not video_path.exists():
        print(f"[❌] Không tìm thấy video: {video_path}")
        return

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"[❌] Không mở được video: {video_path}")
        return

    history: Deque[Tuple[int, int]] = deque(maxlen=history_len)
    print("[INFO] Nhấn 'q' để thoát.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(
            source=frame,
            imgsz=768,
            conf=CONF_THRESH,
            verbose=False,
        )

        if len(results) == 0:
            num_open, num_closed = 0, 0
        else:
            num_open, num_closed = decode_counts_from_result(results[0])

        closed_ratio, total_open, total_closed = update_drowsy_history(
            history, num_open, num_closed, maxlen=history_len
        )
        total = total_open + total_closed
        is_drowsy = (total >= MIN_EYES_PER_FRAME) and (closed_ratio >= drowsy_thresh)

        plotted = results[0].plot() if len(results) > 0 else frame.copy()
        plotted = draw_info(plotted, num_open, num_closed, closed_ratio, is_drowsy)

        cv2.imshow("Drowsy Detection - Video", plotted)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("[✅] Đã xong video.")


# ===========================
# MAIN: MENU ĐƠN GIẢN (KHÔNG CẦN CLI)
# ===========================

def main():
    """
    Chạy trực tiếp file (VS Code Run) → hiện menu:
      1. Webcam
      2. Ảnh
      3. Folder ảnh
      4. Video
    """
    model = load_model(BEST_MODEL)

    print("\n====================")
    print("  DROWSY DETECTION  ")
    print("====================")
    print("1) Webcam realtime")
    print("2) Phân tích 1 ảnh")
    print("3) Phân tích 1 folder ảnh")
    print("4) Phân tích 1 video")
    print("0) Thoát")
    choice = input("Chọn mode (0-4): ").strip()

    if choice == "1":
        idx = input("Nhập camera index (mặc định 0): ").strip()
        cam_idx = int(idx) if idx else 0
        run_webcam(model, cam_index=cam_idx)
    elif choice == "2":
        path = input("Nhập đường dẫn ảnh: ").strip()
        analyze_image(model, path)
    elif choice == "3":
        folder = input("Nhập folder ảnh: ").strip()
        analyze_folder(model, folder)
    elif choice == "4":
        path = input("Nhập đường dẫn video: ").strip()
        analyze_video(model, path)
    else:
        print("Thoát.")


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'ultralytics'